<a href="https://colab.research.google.com/github/prateek-joshi/Pedestrian-Counter/blob/master/people_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os

MODEL_NAME = 'Centernet_MobileNet_V2_FPN_512x512'
PRETRAINED_MODEL_NAME = 'centernet_mobilenetv2fpn_512x512_coco17_od'
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz'
TFRECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [2]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',MODEL_NAME, 'export'), 
#     'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
#     'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

In [3]:
files_paths = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TFRECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [4]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# Install tensorflow object-detection API

In [5]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 58151, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 58151 (delta 266), reused 401 (delta 200), pack-reused 57672
Receiving objects: 100% (58151/58151), 573.07 MiB | 36.38 MiB/s, done.
Resolving deltas: 100% (40252/40252), done.


In [ ]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

In [7]:
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
# !python {VERIFICATION_SCRIPT}

# Download Images

In [ ]:
# Download the data using the kaggle API

from google.colab import files
files.upload() #upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d karthika95/pedestrian-detection
!unzip /content/pedestrian-detection.zip -d {paths['IMAGE_PATH']}

# Download pre-trained model

In [9]:
if os.name=='posix':
    !wget {MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-06-09 13:43:50--  http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.98.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.98.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44016454 (42M) [application/x-tar]
Saving to: ‘centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz’

centernet_mobilenet 100%[===================>]  41.98M   119MB/s    in 0.4s    

2021-06-09 13:43:51 (119 MB/s) - ‘centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz’ saved [44016454/44016454]

centernet_mobilenetv2_fpn_od/
centernet_mobilenetv2_fpn_od/model.tflite
centernet_mobilenetv2_fpn_od/saved_model/
centernet_mobilenetv2_fpn_od/saved_model/saved_model.pb
centernet_mobilenetv2_fpn_od/saved_model/variables/
centernet_mobilenetv2_fpn_od/saved_model/variables/variables.index
centernet_mobilenetv2_

In [10]:
# Create labelmap

labels = [
          {'name':'person', 'id':1},
          {'name':'person-like', 'id':2}
]

with open(files_paths['LABELMAP'], 'w+') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Generate tfRecords

In [12]:
files.upload()  # Upload xml_to_csv.py and generate_tfrecord.py
!mv xml_to_csv.py {paths['SCRIPTS_PATH']}
!mv generate_tfrecord.py {paths['SCRIPTS_PATH']}

Saving xml_to_csv.py to xml_to_csv.py


In [21]:
# Convert all annotations.xml to labels.csv
!python Tensorflow/scripts/xml_to_csv.py

Successfully converted xml to csv - train_labels.csv
Successfully converted xml to csv - test_labels.csv


In [22]:
# Change the extensions of filenames in the csv files
import pandas as pd

def replace_extensions(df):
  df_new = df.copy()
  df_new['filename'] = df_new['filename'].str.replace('.xml','.jpg')
  return df_new

train_df = pd.read_csv('Tensorflow/workspace/annotations/train_labels.csv')
test_df = pd.read_csv('Tensorflow/workspace/annotations/test_labels.csv')

new_train_df = replace_extensions(train_df)
new_test_df = replace_extensions(test_df)

new_train_df.to_csv('Tensorflow/workspace/annotations/train_labels.csv')
new_test_df.to_csv('Tensorflow/workspace/annotations/test_labels.csv')

In [23]:
# Generate TFRecords using the downloaded script
!python Tensorflow/scripts/generate_tfrecord.py --csv_input=Tensorflow/workspace/annotations/train_labels.csv  --output_path=Tensorflow/workspace/annotations/train.record --image_dir=Tensorflow/workspace/images/Train/Train/JPEGImages
!python Tensorflow/scripts/generate_tfrecord.py --csv_input=Tensorflow/workspace/annotations/test_labels.csv  --output_path=Tensorflow/workspace/annotations/test.record --image_dir=Tensorflow/workspace/images/Test/Test/JPEGImages

2021-06-09 14:03:17.729359: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/Tensorflow/workspace/annotations/train.record
2021-06-09 14:03:21.077147: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/Tensorflow/workspace/annotations/test.record
